In [46]:
import pandas as pd


In [ ]:
raw_oc_release = pd.read_excel('oc release 2.1 and 2.2.xlsx')  # specify the actual sheet name
raw_oc_release.head(3)

In [ ]:
# Split the 'second_column' by new line
raw_oc_release['[SA_only] Link to Use Case'] = raw_oc_release['[SA_only] Link to Use Case'].str.split('\n')
pd.set_option('display.max_colwidth', None)
raw_oc_release.head(3)

In [ ]:
# drop where [SA_only] Link to Use Case is empty
raw_oc_release.dropna(subset=['[SA_only] Link to Use Case'], inplace=True)
raw_oc_release.head(3)


In [ ]:
for index, row in raw_oc_release.iterrows():
    print(row['[SA_only] Link to Use Case'])

In [ ]:
# Use explode to duplicate the first column and expand the second column
oc_release = raw_oc_release.explode('[SA_only] Link to Use Case')

oc_release.head(5)

In [ ]:
# Replace text in the column using regular expression to extract the pattern U-xxxxx
oc_release['[SA_only] Link to Use Case'] = oc_release['[SA_only] Link to Use Case'].str.extract(r'(U-\d+)')
oc_release_exploded = oc_release
oc_release_exploded.head(5)


In [ ]:
# Load the Excel files into dataframes
raw_fl = pd.read_excel('FL Release 2.1.1.xlsx', header=0)  # specify the actual sheet name
raw_fl

In [ ]:
raw_fl['[SA_only] Link to Use Case'] = raw_fl['UID']
raw_fl

In [ ]:
# Ensure there is a common column to merge on, here assuming both have a 'UID' or similar
# Adjust the names if needed according to your DataFrame structures
raw_fl['[SA_only] Link to Use Case'] = raw_fl['UID']  # or another column that matches

# Merge data from raw_fl to oc_release based on the 'UID'
oc_release = oc_release.merge(raw_fl[['[SA_only] Link to Use Case', 'FIP']], on='[SA_only] Link to Use Case', how='left')

# Move the 'FIP' column to the 4th position
columns = list(oc_release.columns)
# Assuming index is zero-based, the 4th position is index 3
columns.insert(3, columns.pop(columns.index('FIP')))
oc_release = oc_release[columns]

# Display the modified DataFrame
oc_release.head()

In [56]:
oc_release['FIP'] = oc_release['FIP'].str.split('\n')

In [ ]:
oc_release

In [ ]:
# Function to check if any of the criteria match
def check_fip_criteria(fip_list):
    if not isinstance(fip_list, list):
        return False
    # Convert all list elements to lowercase for consistent comparison
    fip_list_lower = [fip_item.lower() for fip_item in fip_list]
    # Define the criteria to look for
    criteria = [
        "release 2.1 - yes", "release 2.1 - modified", 
        "release 2.1 appendix - yes", "release 2.1 appendix - modified"
    ]
    # Check if any of the criteria are present in the list
    return any(criterion in fip_list_lower for criterion in criteria)

# Apply the check to each row in the FIP column
oc_release['Is FIP 2.1'] = oc_release['FIP'].apply(lambda x: 'Yes' if check_fip_criteria(x) else '')

# Insert the new column next to the FIP column
fip_index = oc_release.columns.get_loc('FIP')
oc_release.insert(fip_index + 1, 'Is FIP 2.1', oc_release.pop('Is FIP 2.1'))

# Display the DataFrame to verify the new column
oc_release.head()

In [ ]:
# Assume oc_release is your existing DataFrame

# Function to determine if all values are "Yes" for a column in a grouped DataFrame
def all_yes(series):
    return 'Yes' if all(value == 'Yes' for value in series) else 'No'

# Group by 'UID' and apply the function to 'Is FIP 2.1'
is_oc_release_2_1 = oc_release.groupby('UID')['Is FIP 2.1'].apply(all_yes)

# Map the results back to your original DataFrame
oc_release['Is OC Release 2.1'] = oc_release['UID'].map(is_oc_release_2_1)

# Display the DataFrame to verify the new column
oc_release.head()

In [64]:
#export to excel
oc_release.to_excel("withfip.xlsx")

In [ ]:
# Filter the DataFrame to find UIDs where 'Is OC Release 2.1' is 'Yes'
unique_uids_all_yes = oc_release[oc_release['Is OC Release 2.1'] == 'Yes']['UID'].unique()

# Convert the array of unique UIDs into a comma-separated string
unique_uids_all_yes_str = ', '.join(unique_uids_all_yes)

print("Candidates for Product Release 2.1:")
print("# of UCs:", len(unique_uids_all_yes), unique_uids_all_yes_str)